In [1]:
import pandas as pd

In [2]:
crimes = pd.read_csv('../../data/recorded_crimes.csv')
liv = pd.read_csv('../preprocessed/processed_livability.csv')

In [3]:
liv.head()

,Neighbourhoods,2014,2015,2016,2017,2018,2019,2020
0,Valkenberg,6,6,6,6,6,6,7
1,Chassé,6,7,7,7,7,7,8
2,Fellenoord,2,3,2,3,2,3,5
3,Schorsmolen,2,3,2,3,2,3,5
4,Station,6,6,6,6,6,6,6


In [4]:
crimes.head(13)

,ID,SoortMisdrijf,WijkenEnBuurten,Perioden,GeregistreerdeMisdrijven_1
0,437512,Totaal misdrijven,Breda,2012 januari,1382
1,437513,Totaal misdrijven,Breda,2012 februari,1284
2,437514,Totaal misdrijven,Breda,2012 maart,1375
3,437515,Totaal misdrijven,Breda,2012 april,1341
4,437516,Totaal misdrijven,Breda,2012 mei,1455
5,437517,Totaal misdrijven,Breda,2012 juni,1439
6,437518,Totaal misdrijven,Breda,2012 juli,1256
7,437519,Totaal misdrijven,Breda,2012 augustus,1447
8,437520,Totaal misdrijven,Breda,2012 september,1430
9,437521,Totaal misdrijven,Breda,2012 oktober,1482


In [5]:
crimes['Perioden'] = crimes['Perioden'].str[:4].str.strip()
crimes['Perioden']

0         2012
1         2012
2         2012
3         2012
4         2012
          ... 
545627    2022
545628    2023
545629    2023
545630    2023
545631    2023
Name: Perioden, Length: 545632, dtype: object

In [6]:
grouped_df = crimes.groupby(['Perioden', 'WijkenEnBuurten', 'SoortMisdrijf']).sum()['GeregistreerdeMisdrijven_1']
grouped_df = grouped_df.reset_index()


In [7]:
grouped_df2 = grouped_df[grouped_df['GeregistreerdeMisdrijven_1'] != 0]

In [8]:
grouped_df2 = grouped_df2[grouped_df2['WijkenEnBuurten'] != 'Breda']
grouped_df2 = grouped_df2[grouped_df2['SoortMisdrijf'] == 'Totaal misdrijven']

In [9]:
grouped_df2 = grouped_df2.sort_values('Perioden')

In [10]:
grouped_df2 = grouped_df2.loc[grouped_df['Perioden'] > str(2013)]

In [11]:
melted_df = liv.melt(id_vars=["Neighbourhoods"],
                     var_name="Year",
                     value_name="Livability index")

In [12]:
merged_dataset = pd.merge(grouped_df2, melted_df, left_on='WijkenEnBuurten', right_on='Neighbourhoods')

In [13]:
asd = merged_dataset.loc[merged_dataset['Year'] == merged_dataset['Perioden']]

In [14]:
asd2 = asd.drop('Perioden', axis=1)
asd2 = asd2.drop('WijkenEnBuurten', axis=1)


In [15]:
asd2.to_csv('merged_cr_liv.csv', index=False)


In [16]:
merged_b = pd.read_csv('../../data/green_index_move_houses_public_nuisance.csv')

In [17]:
asd2['Year'] = asd2['Year'].astype(int)

In [18]:
final2 = pd.merge(merged_b, asd2, left_on=['year', 'Neighbourhood'], right_on=['Year', 'Neighbourhoods'])

In [21]:
final2 = final2.drop('year', axis=1)
final2 = final2.drop('Neighbourhoods', axis=1)

In [22]:
final2

,Neighbourhood,green_score,frequency,GeregistreerdeOverlast_1,SoortMisdrijf,GeregistreerdeMisdrijven_1,Year,Livability index
0,Bavel,28.123801,1.583333,1.863248,Totaal misdrijven,439,2014,7
1,Bavel,28.123801,1.592593,1.982906,Totaal misdrijven,434,2015,7
2,Bavel,30.643434,1.577236,1.829060,Totaal misdrijven,427,2016,7
3,Bavel,33.500000,1.525000,1.658120,Totaal misdrijven,350,2017,7
4,Bavel,30.423769,1.693333,1.401709,Totaal misdrijven,334,2018,7
...,...,...,...,...,...,...,...,...
382,Zandberg,26.731392,4.374046,2.974359,Totaal misdrijven,318,2016,9
383,Zandberg,20.985215,4.328125,2.632479,Totaal misdrijven,258,2017,9
384,Zandberg,19.118199,4.441860,3.316239,Totaal misdrijven,237,2018,9
385,Zandberg,18.183772,4.453782,2.837607,Totaal misdrijven,268,2019,9


In [23]:
final2.to_csv('../final/final_merge.csv', index=False)